<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/nb/censor/examples/Censor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install videodb openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

In [11]:
from videodb import connect 
conn = connect()

In [12]:
coll = conn.get_collection()
video = conn.upload(url="https://www.youtube.com/watch?v=NFTtDY0efF0")

In [15]:
video.index_scenes()

In [16]:
scenes = video.get_scenes()
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

In [42]:
llm_prompt = """Here’s the data from a vision (scene) index run on a video footage. The structure of the data is { - index: scene description}. 
Based on this information, identify and filter the scenes safe for mature audiences but do not include nudity.
As a response, return a json which contains one fields, safe and its value is a list of all indexes(numbers) that are safe \n \n"""

for index, scene in enumerate(scenes):
    llm_prompt += f"{index}: {scene['response']}\n"

In [48]:
import openai
import json

client = openai.OpenAI()

openai_res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": llm_prompt}],
    response_format={"type": "json_object"}
)
openai_res = json.loads(openai_res.choices[0].message.content)

In [61]:
shots = []
for i in openai_res["safe"]:
    shots.append([float(scenes[i]['start']), float(scenes[i]['end'])])

print(shots)

In [86]:
def merge_timestamps(shots, threshold):
    merged_shots = [shots[0]]  

    for current_start, current_end in shots[1:]:
        last_end = merged_shots[-1][1]

        # If the current start time is within the threshold of the last end time, merge them.
        if current_start - last_end <= threshold:
            merged_shots[-1][1] = max(last_end, current_end)  # Update the end time of the last interval.
        else:
            # Otherwise, add the current interval as a new entry.
            merged_shots.append([current_start, current_end])

    return merged_shots 


merged_shots = merge_timestamps(shots, 1)

In [ ]:
from videodb import play_stream

stream_url = video.generate_stream(timeline=merged_shots)
play_stream(stream_url)